# TP N° 2
## Obteniendo un posible CUIT a partir de una fecha de nacimiento y género

### Librerias necesarias

In [9]:
#Libreria para tratar con Arrays y otro tipos de datos utilizados
import numpy as np
#Libreria para graficar
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#Libreria que probee herramientas para el manejo de datos
import pandas as pd
#Libreria que incluye redes neuronales, en nuestro caso un MLP (Multi Layer Preceptron)
from sklearn.neural_network import MLPRegressor
#Libreria de hora y fecha
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
#Librerias para realizar preprocesado de datos (Transformaciones y estandarizacion)
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler 
plt.style.use('seaborn-poster')
%matplotlib inline

### Cargando el archivo CSV en memoria

In [10]:
#Cargamos el CSV en memoria
cuiles = pd.read_csv('cuiles2.csv', names = ["CUIL", "Nacimiento", "Sexo"])



### Creando la instancia de regresor utilizando un MLP (Multi Layer Perceptron)

#### Multi Layer Perceptron (Perceptrón Multicapa)
Utilizamos un tipo de red neuronal llamado Perceptrón Multicapa, el mismo consta de 4 capas, compuestas por 200 perceptrones para la entrada, 400 y 400 en sus capas ocultas, y 200 en sus capas de salida.
Utilizamos el algoritmo adam (https://arxiv.org/abs/1412.6980) a que provee una excelente respuesta para datasets con mas de 100 registros; cuya funcion de activación es la de rectificador (ReLU), es decir sus valores van desde 0 a f(x) = max(x) es decir, el valor maximo del dataset de entrada.
Esto es particularmente útil dado que probando otro tipo de funciones, como la tangente hiperbolica, se obtinen resultados escalados y suponemos se requeriria una especie de postproceso para reconvertir la salida al dominio de la entrada. Con ReLU nos aseguramos conservar la escala.

In [37]:
#Creamos el objeto MLP con las opciones definidas en su constructor
mlp = MLPRegressor(hidden_layer_sizes=(200,400,400,200), max_iter = 2000, solver='adam', \
                   alpha=0.01, activation = 'relu', random_state = 9)


x = cuiles['Nacimiento']
z = cuiles['Sexo']
y = cuiles['CUIL']

#Convertimos z en 1 y 0
le = preprocessing.LabelEncoder()
le.fit(z)
z = le.transform(z)

#Ponemos cantidad de dias
x =  [((datetime.now() - datetime.strptime(item, '%d-%m-%Y')).days) for item in cuiles['Nacimiento']]

#Creamos la matriz de features, es decir [[Cantidad de dias pasados , Sexo]]
X = np.column_stack((x,z))
X

array([[12366,     1],
       [16476,     0],
       [15689,     0],
       ..., 
       [23534,     0],
       [15788,     0],
       [19723,     0]])

#### Tratamiento a los datos de entrada
Los datos de entrada constan de una matriz de 2xN formada por [Cantidad de dias, Sexo], utilizamos
cantidad de días dado que debemos darle una entrada representativa a la red neuronal, que pueda aprender.
No podemos usar un dato tipo cadena de texto dado que no guardaria una relacion con el dominio del problema.

La columna Sexo, tambien es estandarizada a 0 y 1, siendo 0 Femenino y 1 Hombre.

#### Probando la red neuronal

In [39]:
#Creamos un vector de features que deseamos evaluar para obtener un posible CUIT
predecir = [[((datetime.now() - datetime.strptime('09-07-1989', '%d-%m-%Y')).days), le.transform(['M'])[0]]]

In [40]:

#MLP resultó tras muchísimas pruebas ser muy sensible a las diferentes escalas y rangos de datos
#Por eso realizamos un escalamiento de -1 a 1 en la matriz de entrada
scaler = StandardScaler()  
scaler.fit(X)  
X = scaler.transform(X)


#print(y)
#y = y.tolist()
#scaler = StandardScaler()  
#scaler.fit(y)  
#y = scaler.transform(y)


predecir = scaler.transform(predecir)


mlp = mlp.fit(X, y)
yfit = mlp.predict(predecir)




/home/ezequiel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


#### Resultados
Deseamos predecir el vector: [ 09-07-1989 , M ]

In [41]:
print(predecir)

[[-1.60046228  1.56090554]]


Y obtenemos

In [42]:
print(int(yfit[0]))

20979738158


Valor real: 20345553062